# Zadanie 1
Pobieraj dane z pliku https://github.com/przem85/statistics/blob/master/D8/ANOVA4.txt
Zawiera on dane z eksperymentu na roślinach, które były hodowane w trzech  różnych warunki wzrostu. 

- Wykonaj ANOVA
- Czy trzy grupy są różne?
- Wykonaj analizę post hoc, który z par jest inny? 
- Czy porównanie nieparametryczne (Kruskal-Wallis test) prowadzi do innego wyniku? 

In [53]:
import numpy as np
import scipy.stats as stats
import pandas as pd

In [54]:
df = pd.read_csv('F:\SZKOLENIE\data\ANOVA4.txt',sep=" ",header=None, names=['group','weight'])
df.head()

,group,weight
0,Control,4.17
1,Control,5.58
2,Control,5.18
3,Control,6.11
4,Control,4.50


In [55]:
 # Sort them into groups, according to column 1
data=np.array(df)
group1 = data[data[:,0]=='Control',1]
group2 = data[data[:,0]=='TreatmentA',1]
group3 = data[data[:,0]=='TreatmentB',1]

In [56]:
g_a = df['weight'][df['group']=='TreatmentA']
g_b = df['weight'][df['group']=='TreatmentB']
g_c = df['weight'][df['group']=='Control']

In [57]:
group3

array([6.31, 5.1, 5.54, 5.5, 5.37, 5.9, 4.9, 6.15, 5.8, 5.6], dtype=object)

## Saprawdzamy założenie o równości variancii 

In [58]:
# First, check if the variances are equal, with the "Levene"-test
(W,p) = stats.levene(group1, group2, group3)
if p<0.05:
    print(('Warning: the p-value of the Levene test is <0.05: p={0}'.format(p)))
else:    
    print(('OK: the p-value of the Levene test is >0.05: p={0}'.format(p)))

OK: the p-value of the Levene test is >0.05: p=0.32927821561008164


## Wykonuje ANOWE jednoczynnikową 

In [7]:
# Do the one-way ANOVA
F_statistic, pVal = stats.f_oneway(group1, group2, group3)

In [8]:
print((F_statistic, pVal))
if pVal < 0.05:
    print('One of the groups is significantly different.')

(4.846087862380136, 0.015909958325622899)
One of the groups is significantly different.


## Bardziej szczegółowym wynikiem ANOVA można otrzymać za pomom modelowania statystycznego:

In [60]:
data1=data
data1[data1[:,0]=='Control',0]=1.
data1[data1[:,0]=='TreatmentA',0]=2.
data1[data1[:,0]=='TreatmentB',0]=3.
data1[:,0].astype(float)
data1[:,1].astype(float)
df1=pd.DataFrame(data1, columns=['group','weight'])
df1.head()

,group,weight
0,1,4.17
1,1,5.58
2,1,5.18
3,1,6.11
4,1,4.5


In [64]:
# First, check if the variances are equal, with the "Levene"-test
(W,p) = stats.levene(group1, group2, group3)
if p<0.05:
    print(('Warning: the p-value of the Levene test is <0.05: p={0}'.format(p)))
else:
    print(('OK: the p-value of the Levene test is >0.05: p={0}'.format(p)))

OK: the p-value of the Levene test is >0.05: p=0.32927821561008164


In [65]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
# Elegant alternative implementation, with pandas & statsmodels
model = ols('weight ~ C(group)', df).fit()
anovaResults = anova_lm(model)
print(anovaResults)


            df     sum_sq   mean_sq        F    PR(>F)
C(group)   2.0   4.663727  2.331863  6.00568  0.006956
Residual  27.0  10.483460  0.388276      NaN       NaN


D:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
D:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
D:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1814: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [67]:
from statsmodels.stats import multicomp
mc = multicomp.MultiComparison(df['weight'], df['group'])
print(mc.tukeyhsd().summary())

# Show the group names


 Multiple Comparison of Means - Tukey HSD,FWER=0.05 
  group1     group2   meandiff  lower  upper  reject
----------------------------------------------------
 Control   TreatmentA  -0.373  -1.0638 0.3178 False 
 Control   TreatmentB  0.585   -0.1058 1.2758 False 
TreatmentA TreatmentB  0.958    0.2672 1.6488  True 
----------------------------------------------------


In [68]:
print('\n Kruskal-Wallis test ----------------------------------------------------')


# Then do the Kruskal-Wallis test
h, p = stats.kruskal(g_c, g_a, g_b)
print('Result from Kruskal-Wallis test: p = {0}'.format(p))


 Kruskal-Wallis test ----------------------------------------------------
Result from Kruskal-Wallis test: p = 0.011030052001273464
